In [ ]:
import os
import sys

In [ ]:
sys.path.append('../input/kerasdeeplabv3pluslatest/keras-deeplab-v3-plus-master')

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os
from tqdm import tqdm
from tensorflow.keras.layers import Convolution2D,ZeroPadding2D,Conv2D,MaxPool2D,Dense,BatchNormalization,Dropout,Input,Activation,Flatten,Add,UpSampling2D,MaxPooling2D,Concatenate,AveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
import sys
from multiprocessing.dummy import Pool as ThreadPool
import random
import numpy as np
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import os
import time
from skimage.transform import resize
from model import Deeplabv3
from tensorflow.keras.losses import binary_crossentropy

In [ ]:
img=tf.keras.preprocessing.image.load_img('../input/massachusetts-roads-dataset/road_segmentation_ideal/training/input/img-10.png')
plt.imshow(img)

In [ ]:
mask=tf.keras.preprocessing.image.load_img('../input/massachusetts-roads-dataset/road_segmentation_ideal/training/output/img-10.png')
plt.imshow(mask)
np.unique(np.array(mask))

In [ ]:
# Reading the dataset

filenames_inp=pd.DataFrame({"file_names":os.listdir("../input/massachusetts-road-512-patches/massachusetts-roads-dataset-patches/training/input")})
filenames_output=pd.DataFrame({"file_names":os.listdir("../input/massachusetts-road-512-patches/massachusetts-roads-dataset-patches/training/output")})
filenames=filenames_inp.loc[filenames_inp.file_names.isin(filenames_output["file_names"]),:]

In [ ]:
len(filenames)

In [ ]:
# Reading the dataset

val_filenames_inp=pd.DataFrame({"file_names":os.listdir("../input/massachusetts-road-512-patches/massachusetts-roads-dataset-patches/testing/input")})
val_filenames_output=pd.DataFrame({"file_names":os.listdir("../input/massachusetts-road-512-patches/massachusetts-roads-dataset-patches/testing/output")})
val_filenames=val_filenames_inp.loc[val_filenames_inp.file_names.isin(val_filenames_output["file_names"]),:]

In [ ]:
len(filenames_output)

In [ ]:
len(val_filenames)

In [ ]:
NUM_PARALLEL_CALLS_DS=os.cpu_count()

In [ ]:
from tensorflow.keras import backend as K
# Creating the dataset

def get_image_data(image_name):
    '''
        This Function is used to read stored patches of images as numpy array and convert them to tensor
    '''

    image_name_str=image_name.numpy().decode('utf8')
    img = np.load('../input/massachusetts-road-512-patches/massachusetts-roads-dataset-patches/training/input/' + image_name_str,allow_pickle=True)
    msk = np.load('../input/massachusetts-road-512-patches/massachusetts-roads-dataset-patches/training/output/' + image_name_str,allow_pickle=True)

    random_num = random.choice([0,1,2])    
    fun_dict={0:lambda x:x,
              1:tf.image.flip_left_right,
              2:tf.image.flip_up_down}
    aug_fun=fun_dict[random_num]
    img=aug_fun(img)
    msk=aug_fun(msk)
    return tf.cast(tf.image.resize(img,(512,512)),tf.float32)/255.,tf.image.resize(tf.cast(msk,tf.float32)/255.,(512,512),method='nearest')

def get_image_data_val(image_name):
    '''
        This Function is used to read stored patches of images as numpy array and convert them to tensor
    '''
    image_name_str=image_name.numpy().decode('utf8')
    img = np.load('../input/massachusetts-road-512-patches/massachusetts-roads-dataset-patches/testing/input/' + image_name_str,allow_pickle=True)
    msk = np.load('../input/massachusetts-road-512-patches/massachusetts-roads-dataset-patches/testing/output/' + image_name_str,allow_pickle=True)
    img=tf.cast(tf.image.resize(img,(512,512)),tf.float32)/255.
    msk=tf.image.resize(tf.cast(msk,tf.float32)/255.,(512,512),method='nearest')    
    return img,msk

def preprocessing(x_inp):
    """
        Pyfunction for training set
    """
    img,msk=tf.py_function( get_image_data, [x_inp] , Tout=(tf.float32,tf.float32))
    img.set_shape([512,512,3])
    msk.set_shape([512,512,1])
    return img,msk

def preprocessing_val(x_inp):
    """
        Pyfunction for validation set
    """
    img,msk=tf.py_function( get_image_data_val, [x_inp] , Tout=(tf.float32,tf.float32))
    img.set_shape([512,512,3])
    msk.set_shape([512,512,1])
    return img,msk

with tf.device('/cpu:0'):
    train_data=tf.data.Dataset.from_tensor_slices(filenames['file_names']).shuffle(buffer_size=len(filenames)).map(preprocessing,num_parallel_calls=NUM_PARALLEL_CALLS_DS).batch(8)
    validation_data=tf.data.Dataset.from_tensor_slices(val_filenames['file_names']).map(preprocessing_val,num_parallel_calls=NUM_PARALLEL_CALLS_DS).batch(8)

In [ ]:
from tensorflow.keras import backend as K
K.sum(tf.cast((tf.constant([2,1,1,0])!=0),tf.float32))

In [ ]:
def iou_fun(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    num = K.sum(K.abs(y_true * tf.math.round(y_pred)))
    den = K.sum(K.abs(tf.cast((y_true + tf.math.round(y_pred))!=0,tf.float32)))    
    return num/den

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def iou_fun(y_true, y_pred):
    """
      This function calculated IOU
    """
    num = K.sum(K.abs(y_true * tf.math.round(y_pred)))
    den = K.sum(K.abs(tf.cast((y_true + tf.math.round(y_pred))!=0,tf.float32)))    
    return num/den

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_predict):
    return binary_crossentropy(y_true, y_predict) + (1-dice_coef(y_true, y_predict))



model = Deeplabv3(input_shape=(512, 512, 3), classes=1,activation='sigmoid',backbone='xception')  

iou=tf.keras.metrics.MeanIoU(num_classes=2)
auc_pr=tf.keras.metrics.AUC(num_thresholds=50,curve='PR')
giou_loss=tfa.losses.GIoULoss(mode='iou')

precision=tf.keras.metrics.Precision()
recall=tf.keras.metrics.Recall()
model.compile(optimizer = Adam(lr = 0.0001), loss = bce_dice_loss,  metrics = ['accuracy',iou_fun,precision,recall])
model.summary()

In [ ]:
!mkdir model

In [ ]:
INITIAL_EPOCH=0

save_model_chkpnt=tf.keras.callbacks.ModelCheckpoint(os.path.join('model/model{epoch:02d}.hdf5'),
                                                     monitor='val_accuracy',
                                                     verbose=0,
                                                     save_best_only=True,
                                                     save_weights_only=True,
                                                     save_freq='epoch')

# Callback to reduce Learning Rate
reduce_on_plateau=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_iou_fun', factor=0.1, patience=5, verbose=1,
                                    mode='max', min_delta=0.05)

logger=tf.keras.callbacks.CSVLogger(
    os.path.join('log.csv'), separator=',', append=True
)


if INITIAL_EPOCH!=0:    
    model_final.load_weights('./model/model05.hdf5')    
    model_final.summary()    

In [ ]:
model.fit(train_data,
               validation_data=validation_data,
               initial_epoch=INITIAL_EPOCH,
               callbacks=[save_model_chkpnt,reduce_on_plateau,logger],
               epochs=15)